In [ ]:
import numpy as np
import pandas as pd
import statsmodels.tsa.seasonal as STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from IPython.display import display

testData_01 = pd.read_csv("./TestdatenSet1.csv")
print(testData_01)

# Do one with electrity and one with voltage



In [ ]:
# Strom pipeline
testData_01_power_df = testData_01[["Zeitstempel", "Strom"]].copy()
# We will assume for now that NaN is equivalent to the machine just not running; optional: we could try to build a missing value handler 
# based on linear interpolation (does that even make sense with electricity?) or maybe by estimating a value via loess regression
# set up dataframe with timestamp as an index 
testData_01_power_df.rename(columns= {"Zeitstempel": "Timestamp", "Strom": "Power"}, inplace = True)
testData_01_power_df["Power"] = testData_01_power_df["Power"].str.replace(",", ".", regex=False)
testData_01_power_df["Power"] = pd.to_numeric(testData_01_power_df["Power"], errors = "raise") # if errors then its put into an NaN state
testData_01_power_df.dropna(subset=["Power"], inplace = True)
# print(testData_01_power_df)

# setting up index and interpolate to get missing values 
testData_01_power_df["Timestamp"] = pd.to_datetime(testData_01_power_df["Timestamp"])
testData_01_power_df.set_index("Timestamp", inplace=True)
print(testData_01_power_df)

# most_common_time_inseconds = testData_01_power_df.index.to_series().diff().value_counts().head().index[0].total_seconds()
# testData_01_power_df = testData_01_power_df.resample(pd.Timedelta(seconds=most_common_time_inseconds)).mean()
# testData_01_power_df = testData_01_power_df.interpolate(method="time")
# print(testData_01_power_df)
testData_01_power_df.index = (testData_01_power_df.index  - testData_01_power_df.index[0]).total_seconds()
testData_01_power_df.drop(testData_01_power_df.index[-1], inplace=True)
pd.reset_option("all")
display(testData_01_power_df)



In [ ]:
#transformation
testData_01_power_df["Power"] = np.log(testData_01_power_df["Power"])
display(testData_01_power_df)

In [ ]:
# first step stl
testData_01_power_df = testData_01_power_df.sort_index() #does that even make sense? It did although I assumed data was ordered in the csv file
# trend = math.ceil(1.5*60/(1- 1.5/301))
stl_testData_power = STL.STL(testData_01_power_df["Power"], period=60)
res = stl_testData_power.fit()

fig, axes = plt.subplots(4, 1, figsize=(10,6), sharex=True)
axes[0].plot(testData_01_power_df.index, res.observed, color="black")
axes[0].set_ylabel("Power [kW]")
axes[0].set_xlabel("Observed (in ln())")

axes[1].plot(testData_01_power_df.index, res.trend, color="darkseagreen")
axes[1].set_ylabel("Trend kW")

axes[2].plot(testData_01_power_df.index, res.seasonal, color="orangered")
axes[2].set_ylabel("Season kW")

axes[3].plot(testData_01_power_df.index, res.resid, color="darkslateblue")
axes[3].set_ylabel("Residual kW")
axes[3].set_xlabel("Time elapsed in seconds")

plt.tight_layout()
plt.show()

# Detrending as there not really seasons; remember that STL assumes seasonal>=7; reverse log operations for forecasting values
residual_df = pd.DataFrame(np.exp(res.observed - res.resid))
residual_df.rename(columns={0 : "PowerDifference"}, inplace=True)
print(residual_df)

In [ ]:
#stationary testing via ACF and PACF analysis
plot_acf(residual_df["PowerDifference"], lags=100)
plt.show()
plot_pacf(residual_df["PowerDifference"], lags=100)
plt.show()

In [ ]:
# ARIMA
arima_Model = ARIMA(residual_df["PowerDifference"], order=(2,1,1))
arima_Result = arima_Model.fit()
print(arima_Result.summary())


forecast = arima_Result.forecast(steps=50)
print(forecast)

In [ ]:
#actually used ACF and PACF for Hyperparameter Optimization